# Código para el entrenamiento del modelo YOLOv8 ( n, s, m, l, x)

# 0. Memoria ram y capacidad de memoria

Vamos a comprobar que memoria ram y capacidad estamos usando en Colab

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# 1. Chequear si tengo GPU y cual

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


# 2. Importar librerías

Importamos librerías importantes para el proceso

In [2]:
#0. En Colab, primero instala los paquetes y monta tu Drive:  Instalar las librerías necesarias
!pip install ultralytics wandb  # Esto instalará ultralytics y wandb (torch y torchvision generalmente ya vienen instalados en Colab)

import ultralytics
ultralytics.checks()

Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.9/225.8 GB disk)


# 3. Conectar con Google Drive

In [ ]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')           #cambiar la ruta de drive de donde lo extraes

Mounted at /content/drive


#4. Importar archivos

# 4.1  Importar carpetas desde Google Drive si no están en Rar

In [ ]:


# 2. Definir las carpetas en Google Drive que deseas importar
# Por ejemplo, si tienes dos carpetas: una para imágenes y otra para etiquetas
folders_to_copy = [
    "/content/drive/MyDrive/model_rar_640_tiles_labels/Fotos_elegidas"  # Carpeta con imágenes
   # "/content/drive/MyDrive/Tmodel_rar_640_tiles_labels/fotos_elegidas/labels"   # Carpeta con archivos txt
]

# 3. Definir un directorio de destino en Colab donde copiarás estas carpetas
destination_base = "/content/dataset_importado_bueno"
import os
os.makedirs(destination_base, exist_ok=True)

# 4. Copiar cada carpeta desde Drive al directorio local
for folder in folders_to_copy:
    folder_name = os.path.basename(folder)
    dest = os.path.join(destination_base, folder_name)
    # El comando cp -r copia recursivamente la carpeta
    !cp -r "{folder}" "{dest}"

# 5. Verificar que se copiaron las carpetas
!ls -l "{destination_base}"

#4.2.  Descomprimir archivos rar de Drive a Google collab directamente si están en rar

In [ ]:


# 2. Instalar unrar para poder descomprimir archivos RAR
!apt-get update
!apt-get install -y unrar



# 4. Definir las rutas de los archivos RAR en Drive
# Ajusta estas rutas según dónde estén ubicados en tu Drive
images_train_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels/train/images.rar"
labels_train_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels/train/labels.rar"
images_validation_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels/valid/images.rar"
labels_validation_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels//valid/labels.rar"

# 5. Definir las carpetas de destino en Colab para cada uno
# Estas carpetas mantendrán la estructura de YOLO (por ejemplo, "images" y "labels")
dest_train_images = "/content/dataset/train"    # he dejado image sy labels en el mismo directorio a ver si así se apaña el no detectar labels
dest_train_labels = "/content/dataset/train"
dest_val_images = "/content/dataset/val"
dest_val_labels = "/content/dataset/val"

import os
os.makedirs(dest_train_images, exist_ok=True)
os.makedirs(dest_train_labels, exist_ok=True)
os.makedirs(dest_val_images, exist_ok=True)
os.makedirs(dest_val_labels, exist_ok=True)

# 6. Descomprimir cada archivo RAR en su carpeta correspondiente
!unrar x "{images_train_rar}" "{dest_train_images}/"
!unrar x "{labels_train_rar}" "{dest_train_labels}/"
!unrar x "{images_validation_rar}" "{dest_val_images}/"
!unrar x "{labels_validation_rar}" "{dest_val_labels}/"

# Opcional: listar el contenido para confirmar que se descomprimió correctamente
!ls /content/dataset/train/       # he dejado image sy labels en el mismo directorio a ver si así se apaña el no detectar labels
!ls /content/dataset/train/
!ls /content/dataset/val/
!ls /content/dataset/val/


#5. OVERSAMPLING:

Aplicamos oversampling a aquellas fotos con etiquetas .

Aquí elegimos cuantas copias de nuestra foto orginal queremos producir.

El objetivo es aumentar el número de fotos positivas solo para poder tener más que enseñar.

---



In [ ]:
import os
import shutil

# Rutas de las carpetas originales
orig_images_dir = "/content/drive/MyDrive/Fotos_elegidas/train/images.rar"
orig_labels_dir = "/content/drive/MyDrive/Fotos_elegidas/train/labels"

# Carpeta de destino para el dataset balanceado
balanced_dir = "/content/balanced_train"
balanced_images_dir = os.path.join(balanced_dir, "images")
balanced_labels_dir = os.path.join(balanced_dir, "labels")

os.makedirs(balanced_images_dir, exist_ok=True)
os.makedirs(balanced_labels_dir, exist_ok=True)

################################################################
################################################################
# Factor de duplicación para imágenes positivas
dup_factor = 3                                                      ########cuantas veces las quiero multiplicar

#################################################################

# Recorremos todas las imágenes del dataset original
for filename in os.listdir(orig_images_dir):
    if not filename.lower().endswith(('.jpg','.jpeg','.png')):
        continue
    # Ruta de la imagen y de la etiqueta correspondiente
    orig_img_path = os.path.join(orig_images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    orig_label_path = os.path.join(orig_labels_dir, label_filename)

    # Determinar si la imagen es positiva (tiene anotaciones) o negativa
    is_positive = False
    if os.path.exists(orig_label_path):
        with open(orig_label_path, 'r') as f:
            content = f.read().strip()
        if content:  # Si hay algo, es positiva
            is_positive = True

    # Copiar la imagen y el archivo de etiqueta al nuevo directorio
    # Si es positiva, duplicamos según el factor
    copies = dup_factor if is_positive else 1
    for i in range(copies):
        # Para evitar sobreescritura, agregamos un sufijo en el nombre de la imagen duplicada
        new_filename = os.path.splitext(filename)[0]
        if copies > 1:
            new_filename += f"_{i+1}"
        new_filename += os.path.splitext(filename)[1]
        new_img_path = os.path.join(balanced_images_dir, new_filename)
        shutil.copy2(orig_img_path, new_img_path)

        # También copiamos la etiqueta (si existe), ajustando el nombre
        if os.path.exists(orig_label_path):
            new_label_filename = os.path.splitext(label_filename)[0]
            if copies > 1:
                new_label_filename += f"_{i+1}"
            new_label_filename += ".txt"
            new_label_path = os.path.join(balanced_labels_dir, new_label_filename)
            shutil.copy2(orig_label_path, new_label_path)

print("Dataset balanceado creado en:", balanced_dir)


#6. DATA AUGMENTATION

Solo se aplican a fotos con etiquetas.

Produce una imagen para cada transformación (mas la original) , por lo que aumentamos el número de imágenes positivas una vez más. Todo incluyendo etiquetas.  

Voy a combinar este paso con la data augmentation durante el entrenamiento de YOLO.

**IMPORTANTE**
Tengo que saber cuantas imágenes positivas tengo al final y cuantas negativas para poder organizar proporciones


**IMPORTANTE**

Hay dos formas de aplicar Data augmentation:

#6.0 Data Augmentation más completa

Se produce un numero random de imagenes transformadas que incluyes transformaciones aleatorias

puedes controlar numero transformaciones, y numero de imagenes generadas

todo mas la original y label


In [ ]:
import os
import cv2
import albumentations as A
import random
import shutil

def read_yolo_labels(label_path):
    """
    Lee un archivo de etiquetas YOLO y devuelve:
    - bboxes: [x_center, y_center, width, height] (normalizados)
    - classes: Lista de clases correspondiente a cada bbox
    """
    bboxes = []
    classes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    classes.append(parts[0])
                    bbox = list(map(float, parts[1:]))
                    bboxes.append(bbox)
    return bboxes, classes

# 1) Definir una pipeline que escoja aleatoriamente entre 1 y 6 transformaciones
#    con replace=True para permitir tupla en 'n'
transform_pipeline = A.Compose([
    A.SomeOf([
        A.HorizontalFlip(p=1.0),  # voltear horizontalmente
        A.Rotate(limit=(-5, 5), p=1.0),       # rotación aleatoria entre -15° y +15°
        A.Affine(shear=(-5, 5), p=1.0),       # cizallamiento aleatorio entre -15° y +15°
        A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.2),
                                   contrast_limit=(-0.1, 0.2),
                                   p=1.0),
        A.Blur(blur_limit=5, p=1.0),
        A.ColorJitter(brightness=0.05,
                      contrast=0.05,
                      saturation=0.05,
                      #hue=0.2,
                      p=1.0),
    ],
####################################################################################################################################
####################################################################################################################################
    n=(3),        # elige un subconjunto aleatorio de 1 a 6 transformaciones
    replace=True),   # <-- IMPORTANTE: así evitamos la comparación n > len(self.transforms)
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

####################################################################################################################################
####################################################################################################################################

# Directorios de entrada (donde están tus imágenes y anotaciones)
input_images_dir = "/content/drive/MyDrive/fotos_pruebas"
input_labels_dir = "/content/drive/MyDrive/fotos_pruebas"

# Directorios de salida para imágenes positivas
output_images_dir = "/content/dataset_aug_combined4/positives/images"
output_labels_dir = "/content/dataset_aug_combined4/positives/labels"
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Directorios de salida para imágenes negativas
output_neg_images_dir = "/content/dataset_aug_combined4/negatives/images"
output_neg_labels_dir = "/content/dataset_aug_combined4/negatives/labels"
os.makedirs(output_neg_images_dir, exist_ok=True)
os.makedirs(output_neg_labels_dir, exist_ok=True)

# 2) Iterar sobre cada imagen
for filename in os.listdir(input_images_dir):
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(input_images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(input_labels_dir, label_filename)

    bboxes, classes = read_yolo_labels(label_path)

    # 2a) Si no hay anotaciones, se considera "negativa"
    if len(bboxes) == 0:
        # Copiamos la imagen original y un TXT vacío
        shutil.copy2(img_path, os.path.join(output_neg_images_dir, filename))
        open(os.path.join(output_neg_labels_dir, label_filename), 'w').close()
        print(f"{filename}: No tiene anotaciones. Copiada en negativos.")
        continue

    # 2b) Si la imagen es "positiva", copiamos la original
    shutil.copy2(img_path, os.path.join(output_images_dir, filename))
    shutil.copy2(label_path, os.path.join(output_labels_dir, label_filename))

####################################################################################################################################
####################################################################################################################################

    # 3) Generar un número aleatorio de imágenes aumentadas (entre 1 y 10)

    num_variants = random.randint(3, 8)

####################################################################################################################################
####################################################################################################################################
    # 4) Crear las variantes
    for i in range(num_variants):
        # Leer la imagen original en BGR
        image_bgr = cv2.imread(img_path)
        if image_bgr is None:
            print(f"No se pudo leer {filename}, se omite.")
            continue

        # Convertir a RGB para Albumentations
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

        # Aplicar la pipeline
        try:
            result = transform_pipeline(
                image=image_rgb,
                bboxes=bboxes,
                labels=classes
            )
        except Exception as e:
            print(f"Error al aplicar pipeline en {filename}: {e}")
            continue

        aug_img_rgb = result["image"]
        aug_bboxes = result["bboxes"]

        # Convertir de vuelta a BGR para guardar
        aug_img_bgr = cv2.cvtColor(aug_img_rgb, cv2.COLOR_RGB2BGR)

        # Nuevo nombre de archivo
        new_img_name = f"{os.path.splitext(filename)[0]}_aug{i}.jpg"
        new_label_name = f"{os.path.splitext(filename)[0]}_aug{i}.txt"

        # Guardar la imagen
        cv2.imwrite(os.path.join(output_images_dir, new_img_name), aug_img_bgr)

        # Guardar anotaciones en formato YOLO
        with open(os.path.join(output_labels_dir, new_label_name), 'w') as f:
            for cls, bbox in zip(classes, aug_bboxes):
                f.write(f"{cls} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

    print(f"Procesada {filename}: generadas {num_variants} variantes.")

print("Data augmentation completo.")


Procesada DSC02325.jpg: generadas 10 variantes.
Data augmentation completo.


#6.1 Data augmentation total

Se produce una imagen por cada transformación ( es la que más imágenes produce)

In [ ]:
import os
import cv2
import albumentations as A
import shutil

# Función para leer anotaciones YOLO
def read_yolo_labels(label_path):
    bboxes = []
    classes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    classes.append(parts[0])
                    bbox = list(map(float, parts[1:]))
                    bboxes.append(bbox)
    return bboxes, classes


######################################################################################
######################################################################################

#hay dods cosas a cambiar, la p (probabilidad de que ocurra, y los valores entre los que ocurren teniendo en cuenta las data augmentation que pasan en el training)

######################################################################################
######################################################################################




######################################################################################
# Solo transformaciones (sin ToTensorV2):
######################################################################################

pipeline_flip = A.Compose([
    A.HorizontalFlip(p=1.0),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

pipeline_rotate = A.Compose([
    A.Rotate(limit=5, p=1.0),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

pipeline_shear = A.Compose([
    A.Affine(shear=5, p=1.0),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

pipeline_bright_contrast = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=(0, 0.2), contrast_limit=(0, 0.2), p=0.5),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

pipeline_blur = A.Compose([
    A.Blur(blur_limit=5, p=1.0),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

pipeline_no_aug = A.Compose([
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

######################################################################################
# NUEVO PIPELINE: Ajuste de brillo, contraste, saturación y hue (ColorJitter)
######################################################################################

pipeline_color_jitter = A.Compose([
    A.ColorJitter(brightness=0.2,
                  contrast=0.2,
                  saturation=0.2,
                  hue=0.2,
                  p=0.5),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

######################################################################################
# Lista de pipelines (se ha quitado el 'noise')
######################################################################################

pipelines = [
    ("flip", pipeline_flip),
    ("rotate", pipeline_rotate),
    ("shear", pipeline_shear),
    ("bright", pipeline_bright_contrast),
    ("blur", pipeline_blur),
    ("color_jitter", pipeline_color_jitter),
    ("noaug", pipeline_no_aug)
]

def apply_pipeline(image_path, bboxes, labels, pipeline):
    # Leemos la imagen en BGR
    image = cv2.imread(image_path)
    if image is None:
        return None, None

    # Aplicamos Albumentations directamente en BGR
    augmented = pipeline(image=image, bboxes=bboxes, labels=labels)
    aug_img = augmented["image"]   # Esto es un array NumPy en BGR (uint8)
    aug_bboxes = augmented["bboxes"]
    return aug_img, aug_bboxes

######################################################################################
# Directorios
######################################################################################

input_images_dir = "/content/drive/MyDrive/fotos_pruebas"
input_labels_dir = "/content/drive/MyDrive/fotos_pruebas"
output_images_dir = "/content/dataset_aug4/images"
output_labels_dir = "/content/dataset_aug4/labels"

os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

######################################################################################
# Proceso de augmentación
######################################################################################

for filename in os.listdir(input_images_dir):
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    img_path = os.path.join(input_images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(input_labels_dir, label_filename)

    bboxes, classes = read_yolo_labels(label_path)

    # Si no hay anotaciones, copiamos la imagen original
    if len(bboxes) == 0:
        shutil.copy2(img_path, os.path.join(output_images_dir, filename))
        open(os.path.join(output_labels_dir, label_filename), 'w').close()
        continue

    # Aplicamos cada pipeline
    for suffix, pipeline in pipelines:
        aug_img, aug_bboxes = apply_pipeline(img_path, bboxes, classes, pipeline)
        if aug_img is None:
            continue

        new_img_name = os.path.splitext(filename)[0] + f"_{suffix}" + os.path.splitext(filename)[1]
        new_label_name = os.path.splitext(filename)[0] + f"_{suffix}" + ".txt"

        # Guardamos la imagen aumentada
        cv2.imwrite(os.path.join(output_images_dir, new_img_name), aug_img)

        # Guardamos las anotaciones YOLO
        with open(os.path.join(output_labels_dir, new_label_name), 'w') as f:
            for cls, bbox in zip(classes, aug_bboxes):
                f.write(f"{cls} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

    print(f"Procesada {filename} con todos los pipelines.")

print("Data augmentation completo.")



Procesada DSC02325.jpg con todos los pipelines.
Data augmentation completo.


# 6.2 Data Augmentation parcial

Se produce un número random de transformaciones de data augmentation para cada imagen, por tanto, obtienes un número random de imagenes positivas con transformaciones.

(Siempre obtienes original)

In [ ]:
import os
import cv2
import albumentations as A
import random
import shutil

# Función para leer anotaciones en formato YOLO
def read_yolo_labels(label_path):
    """
    Lee un archivo de etiquetas YOLO y devuelve dos listas:
    - bboxes: Lista de bounding boxes en formato [x_center, y_center, width, height] (normalizados).
    - classes: Lista de clases (como string o número) correspondiente a cada bbox.
    """
    bboxes = []
    classes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    classes.append(parts[0])
                    bbox = list(map(float, parts[1:]))
                    bboxes.append(bbox)
    return bboxes, classes

# Lista de transformaciones individuales disponibles
# Se ha eliminado ("noise", A.GaussNoise(...))
# Se ha añadido ("color_jitter", A.ColorJitter(...))
augmentation_transforms = [
    ("flip", A.HorizontalFlip(p=1.0)),
    ("rotate", A.Rotate(limit=15, p=1.0)),      # Rota entre -15° y +15°
    ("shear", A.Affine(shear=15, p=1.0)),       # Aplica shear de 15°
    ("bright", A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0)),
    ("blur", A.Blur(blur_limit=7, p=1.0)),
    # NUEVA TRANSFORMACIÓN PARA BRILLO/CONTRASTE/SATURACIÓN/HUE
    ("color_jitter", A.ColorJitter(
        brightness=0.2,    # ±20% en brillo
        contrast=0.2,      # ±20% en contraste
        saturation=0.2,    # ±20% en saturación
        hue=0.2,           # ±20% en tono
        p=1.0              # se aplica siempre; cámbialo a 0.5 si quieres probabilidad 50%
    ))
]

def create_pipeline_for_transform(transform, target_size=(640, 640)):
    """
    Crea una pipeline de Albumentations que aplica la transformación dada,
    luego redimensiona a target_size. NO se incluye ToTensorV2() para
    evitar la conversión a tensores y conservar la imagen en NumPy.
    """
    return A.Compose([
        transform,
        A.Resize(target_size[0], target_size[1], interpolation=cv2.INTER_LINEAR)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

# Rutas de entrada
input_images_dir = "/content/drive/MyDrive/fotos_pruebas"
input_labels_dir = "/content/drive/MyDrive/fotos_pruebas"

# Rutas de salida para imágenes positivas (con anotaciones)
output_images_dir = "/content/dataset_aug100/positives/images"
output_labels_dir = "/content/dataset_aug100/positives/labels"
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Rutas de salida para imágenes negativas (sin anotaciones)
output_neg_images_dir = "/content/dataset_aug100/negatives/images"
output_neg_labels_dir = "/content/dataset_aug100/negatives/labels"
os.makedirs(output_neg_images_dir, exist_ok=True)
os.makedirs(output_neg_labels_dir, exist_ok=True)

# Iterar sobre cada imagen en el directorio de entrada
for filename in os.listdir(input_images_dir):
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(input_images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(input_labels_dir, label_filename)

    # Leer las anotaciones (bounding boxes y clases)
    bboxes, classes = read_yolo_labels(label_path)

    # Si la imagen no tiene anotaciones, se considera "negativa"
    if len(bboxes) == 0:
        # Copiamos la imagen original a la carpeta de negativos
        shutil.copy2(img_path, os.path.join(output_neg_images_dir, filename))
        # Creamos un archivo TXT vacío en la carpeta de negativos
        open(os.path.join(output_neg_labels_dir, label_filename), 'w').close()
        print(f"{filename}: No tiene anotaciones. Copiada en negativos.")
        continue

    # En caso de imagen "positiva", copiamos la original y sus anotaciones a la carpeta de positivos
    shutil.copy2(img_path, os.path.join(output_images_dir, filename))
    shutil.copy2(label_path, os.path.join(output_labels_dir, label_filename))

    # Determinar de forma aleatoria cuántas variantes se aplicarán (entre 1 y el total de transformaciones)
    num_variants = random.randint(1, len(augmentation_transforms))
    # Seleccionar aleatoriamente 'num_variants' transformaciones (sin combinarlas entre sí)
    selected_transforms = random.sample(augmentation_transforms, num_variants)

    # Para cada transformación seleccionada, se aplica de forma individual
    for suffix, transform in selected_transforms:
        pipeline = create_pipeline_for_transform(transform, target_size=(640, 640))
        try:
            # 1) Leemos la imagen con OpenCV en BGR
            image_bgr = cv2.imread(img_path)
            if image_bgr is None:
                print(f"No se pudo leer {filename}, se omite.")
                continue

            # 2) Convertimos a RGB para que Albumentations maneje bien las transformaciones de color
            image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

            # 3) Aplicamos el pipeline
            result = pipeline(
                image=image_rgb,
                bboxes=bboxes,
                labels=classes
            )

            # La imagen resultante en 'result["image"]' sigue siendo un NumPy array (RGB, uint8)
            aug_img = result["image"]
            aug_bboxes = result["bboxes"]
        except Exception as e:
            print(f"Error al aplicar {suffix} en {filename}: {e}")
            continue

        if aug_img is None:
            continue

        # 4) Convertir de RGB a BGR para guardar de forma consistente
        aug_img_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

        # 5) Generar un nuevo nombre de archivo para esta variante
        new_img_name = os.path.splitext(filename)[0] + f"_{suffix}" + os.path.splitext(filename)[1]
        new_label_name = os.path.splitext(filename)[0] + f"_{suffix}" + ".txt"

        # 6) Guardar la imagen aumentada (BGR) en disco
        cv2.imwrite(os.path.join(output_images_dir, new_img_name), aug_img_bgr)

        # 7) Guardar las anotaciones actualizadas en formato YOLO
        with open(os.path.join(output_labels_dir, new_label_name), 'w') as f:
            for cls, bbox in zip(classes, aug_bboxes):
                f.write(f"{cls} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

        print(f"{filename}: Variante '{suffix}' generada.")

    print(f"Procesada {filename} con {num_variants} variantes aleatorias.")

print("Data augmentation completo: Se han generado las imágenes aumentadas y sus archivos TXT correspondientes para imágenes positivas, y se han copiado las imágenes sin anotaciones en el set de negativos.")


# 6.3 Calcular números archivos carpeta para hacer proporciones imágenes positivas y negativas

In [ ]:
import os

# Rutas de las dos carpetas
folder1 = "/ruta/a/tu/carpeta1"
folder2 = "/ruta/a/tu/carpeta2"

def count_files_in_folder(folder_path):
    """
    Cuenta el número de archivos (no directorios) en la carpeta dada.
    """
    count = 0
    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isfile(full_path):
            count += 1
    return count

# Contar archivos en cada carpeta
count1 = count_files_in_folder(folder1)
count2 = count_files_in_folder(folder2)

# Mostrar resultados
print(f"Carpeta 1: {count1} archivos")
print(f"Carpeta 2: {count2} archivos")
print(f"Total de archivos en cada carpeta: Imágenes {count1} + Labels: {count2}")

#6.4. Guardar resultados en Drive

(en caso de que necesites guardar resultados)

In [ ]:
# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/fotos_pruebas/resultados3"
!mkdir -p "{dest_folder}"                                                           #El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r /content/dataset_aug3 "{dest_folder}/"   #copiar los reultados del modelo en drive


# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"

#7. WANDB

Conectar Weights & Biases para poder seguir el proceso de aprendizaje

Se guarda todo en la nube de la página web

In [3]:

# 8. Autenticación en Weights & Biases (W&B)
import wandb
from wandb.integration.ultralytics import add_wandb_callback

# Initialize your Weights & Biases environment
wandb.login(key="105f813f52ef215433a521087f025ed76bf34c23")  # Te pedirá ingresar tu API key o autenticarte desde el navegador.

# Enable Weights & Biases logging
yolo_settings_wandb=True

# Disable Weights & Biases logging
#yolo settings wandb=False


#8.1 Initialize a Weight and Biases run:
wandb.init(project= "seals_2025 ", job_type= "training")



wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alberto22 (alberto22-university-centre-of-the-westfjords) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# 8. Mejorar la computación de la GPU, somehow

In [ ]:
# 5. Configurar PyTorch para GPU (opcional: optimización extra)
import torch
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # mejora la velocidad en convoluciones si el input es fijo

#9. Importar YOLO


Impotamos el modelo de Yolo que vamos a usar

Aseguramos que WANDb está conectado

In [ ]:

# 9. Ejecutar el entrenamiento de YOLOv8
from ultralytics import YOLO

# Carga el modelo (en este ejemplo, YOLOv8s)
model = YOLO("yolov8s.pt")

#Add W&B callback for ultralytics:
add_wandb_callback(model, True, True)



#10. Importar el data.yaml de drive

una vez tenemos todo en Google colab,  para hacer bien los directorios

In [ ]:
################################IMPORTAR EL DAYA.YAML UNA VEZ TIENES LA ESTRUCTURA  ####################################################
####################################################################################


# Copiar el archivo data.yaml desde Drive a la ruta deseada en Colab
!cp "/content/drive/MyDrive/model_rar_640_tiles_labels/Fotos_elegidas/data.yaml" "/content/dataset_importado_bueno/Fotos_elegidas/data.yaml"

# Verificar que se copió correctamente
!ls -l "/content/dataset"

#11. **Train de model**

El paso importante, entrenar al modelo.

Anotar qué parámetros usas para ver max perfomance y cambiar el data.yaml cuando se requiera

In [ ]:
# Ejecuta el entrenamiento utilizando el archivo data.yaml en la carpeta extraída.
model.train(
    project="seals_2025",
    name="focas_prueba_4_images",
    data="/content/dataset_importado_bueno/Fotos_elegidas/data.yaml",  # Asegúrate de que data.yaml use rutas correctas para Colab
    epochs=100,                                                 # Número total de épocas (pasadas completas por el dataset de entrenamiento)
    imgsz=640,                                                  # Tamaño de la imagen (se reescala a 640x640)
    batch=16,                                                   # Número de imágenes por batch
    lr0=0.001,        #probar 0.0005                            # Tasa de aprendizaje inicial. Un valor mayor hace que los pesos se actualicen más rápido, pero puede inestabilizar el entrenamiento.
    #momentun=0.937,                                            # Momentum para el optimizador (por ejemplo, en SGD)
    optimizer="adamw",       #otra opcion es = sgd              # Se usa AdamW (una variante de Adam que incorpora weight decay para evitar sobreajuste)
    weight_decay=0.0005,                                        # Penalización sobre los pesos para prevenir sobreajuste. Si es muy alto, puede dificultar el aprendizaje; si es muy bajo, el modelo puede sobreajustarse.                                          # Se usa AdamW (una variante de Adam que incorpora weight decay para evitar sobreajuste)
    #augment=False,                                             # Desactiva el data augmentation on the fly de YOLO##############  asi puedo hacer yo la data augmentation con las fotos con labels
    #half=True,                                                 # Activa precisión mixta (FP16) para GPU
    workers=16,                                                 # Número de workers para el DataLoader
    #device='cuda' ,                                            # Fuerza el uso de GPU lo que acelera el entrenamiento
     mosaic=1.0,                                                # Desactivar si las imágenes son muy homogéneas, es un data augmentation process
    resume=True                                                 #PARA seguir entrenamiento donde lo dejaste
)



#12 Gráficas durante aprendizaje en Tensorboard

para poder ver el aprendizaje, llamas a tensorboard

In [ ]:
%load_ext tensorboard

# Iniciar TensorBoard apuntando al directorio donde se guardan los logs
%tensorboard --logdir /content/seals_2025/focas_prueba_4_images


# 13. Guardar resultados del modelo entrenado

Tienes que fijarte donde se está guardando, normalmente algo como **runs**

In [ ]:
# 10. Guardar los resultados del experimento

# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/model_rar_640_tiles_labels/Fotos_elegidas/foquitas"
!mkdir -p "{dest_folder}"                                                           #El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r /content/seals_2025/focas_prueba_4_images "{dest_folder}/"   #copiar los reultados del modelo en drive


# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"                                                                 #verifica que los archivos se han copiado correctamente

# Opcional: comprimir la carpeta de resultados para descargarla a tu PC
!zip -r /content/foquitas.zip "{dest_folder}/focas_prueba_4_images"   #Aquí se comprime la carpeta que acabas de copiar en Drive (la subcarpeta focas_training_prueba1_640_24_02_25 dentro de foquitas) en un archivo ZIP llamado foquitas.zip que se guarda en /content (la carpeta principal del entorno de Colab).

# Descargar el archivo ZIP a tu PC
from google.colab import files
files.download('/content/foquitas.zip')                                             #Este comando abre una ventana de descarga para que puedas bajar el archivo ZIP a tu ordenador.

#14  Validación posterior

Aquí puedes ajustar nivel de confianza e IOU para ver la validación del modelo, a pesar d e que lo haga on the fly durante entrenamiento

In [ ]:
# Cargar el modelo entrenado (ajusta la ruta según corresponda)
model = YOLO("runs/detect/train/weights/best.pt")

# Ejecutar validación especificando el archivo de datos y los umbrales deseados
metrics = model.val(data="data.yaml", iou=0.3, conf=0.25)

# Mostrar las métricas obtenidas en la validación
print("Métricas de validación:", metrics)

#15. Inferencia

Da como resultado, tanto las imagenes con bounding box, como el numero total de elementos por clase

Recuerda conectar DRIVE Y ULTRALYTICS


In [ ]:
import os
import json
import csv
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO

# 1. Cargar el modelo
model_path = "/content/drive/MyDrive/model_rar_640_tiles_labels/Fotos_elegidas/foquitas/focas_prueba_4_images/weights/best.pt"
model = YOLO(model_path)

# 2. Definir carpeta de entrada y salida
input_folder = "/content/drive/MyDrive/model_rar_640_tiles_labels/Fotos_elegidas/test/images"
output_folder = "/content/inference_results_4"
os.makedirs(output_folder, exist_ok=True)

# Dentro del output_folder, creamos dos subcarpetas: with_labels y no_labels
with_labels_folder = os.path.join(output_folder, "with_labels")
no_labels_folder = os.path.join(output_folder, "no_labels")
os.makedirs(with_labels_folder, exist_ok=True)
os.makedirs(no_labels_folder, exist_ok=True)

# 3. Diccionarios para almacenar resultados
results_dict = {}           # Por imagen: { "imagen.jpg": {"seal": 3, "hole": 2} }
global_counts = defaultdict(int)  # Recuento total por clase

# 4. Iterar sobre todas las imágenes en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(input_folder, filename)


###########################################     Cambiar Parámetros    ######################################################
############################################################################################################################

        # Realizar la inferencia (ajusta los parámetros conf, iou, etc. si es necesario)
        results = model.predict(source=img_path, imgsz=640, conf=0.25, iou=0.2, augment=False)

############################################################################################################################
############################################################################################################################

        # Si hay resultados (normalmente 1 por imagen si se pasa una sola imagen al modelo)
        if len(results) > 0:
            result = results[0]

            # 5. Dibujar las detecciones en la imagen
            rendered_img = result.plot()   # Devuelve una imagen (array numpy en RGB)
            # Convertir de RGB a BGR para guardar con cv2
            rendered_bgr = cv2.cvtColor(rendered_img, cv2.COLOR_RGB2BGR)

            # Determinar si hay al menos una detección
            if result.boxes is not None and len(result.boxes) > 0:
                # Sí hay detecciones
                out_img_folder = with_labels_folder
            else:
                # No hay detecciones
                out_img_folder = no_labels_folder

            # Guardar la imagen con bounding boxes en la carpeta correspondiente
            out_img_path = os.path.join(out_img_folder, filename)
            cv2.imwrite(out_img_path, rendered_bgr)

            # 6. Contar las detecciones por clase
            if result.boxes is not None and len(result.boxes) > 0:
                cls_tensor = result.boxes.cls
                cls_indices = cls_tensor.cpu().numpy().tolist()
                class_names = [model.names[int(idx)] for idx in cls_indices]
            else:
                class_names = []

            count_dict = {}
            for cls_name in class_names:
                count_dict[cls_name] = count_dict.get(cls_name, 0) + 1

            # Guardar en results_dict
            results_dict[filename] = count_dict

            # Acumular en global_counts
            for cls_name, cnt in count_dict.items():
                global_counts[cls_name] += cnt

            print(f"Inferencia realizada: {filename}")
        else:
            print(f"No se obtuvieron resultados para: {filename}")

# 7. Guardar los resultados en CSV y JSON

# CSV: tendremos filas de la forma [filename, class, count] para cada imagen,
# luego una fila "TOTAL" por clase
csv_file = os.path.join(output_folder, "inference_counts.csv")
with open(csv_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "class", "count"])

    # Escribir los datos por imagen
    for fname, counts in results_dict.items():
        if len(counts) == 0:
            # Imagen sin detecciones
            writer.writerow([fname, "no_detections", 0])
        else:
            for cls, cnt in counts.items():
                writer.writerow([fname, cls, cnt])

    # Agregar filas de TOTales (una por clase)
    for cls, cnt in global_counts.items():
        writer.writerow(["TOTAL", cls, cnt])

# JSON: contendrá un objeto con "por_imagen" y "totales"
json_data = {
    "por_imagen": results_dict,
    "totales": dict(global_counts)
}

json_file = os.path.join(output_folder, "inference_counts.json")
with open(json_file, "w") as f:
    json.dump(json_data, f, indent=4)

print("Inferencia completada. Resultados guardados en:")
print(f"- {with_labels_folder} -> imágenes con detecciones")
print(f"- {no_labels_folder} -> imágenes sin detecciones")
print(f"- {csv_file}")
print(f"- {json_file}")


#16. Guardar resultados de la Inferencia

In [ ]:
# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/model_rar_640_tiles_labels/Fotos_elegidas/resultadosfinales_2_cambiandoiou"
!mkdir -p "{dest_folder}"                                                           #El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r /content/inference_results_4 "{dest_folder}/"   #copiar los reultados del modelo en drive


# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"                                                                 #verifica que los archivos se han copiado correctamente

# Opcional: comprimir la carpeta de resultados para descargarla a tu PC
!zip -r /content/inferencia_2.zip "{dest_folder}/inference_results_4"   #Aquí se comprime la carpeta que acabas de copiar en Drive (la subcarpeta focas_training_prueba1_640_24_02_25 dentro de foquitas) en un archivo ZIP llamado foquitas.zip que se guarda en /content (la carpeta principal del entorno de Colab).

# Descargar el archivo ZIP a tu PC
from google.colab import files
files.download('/content/inferencia_2.zip')                                             #Este comando abre una ventana de descarga para que puedas bajar el archivo ZIP a tu ordenador.